



# Applied Data Science : Capstone Project - Snacks On Wheels





### Description :
    
This project provide data analysis support for “Snacks on wheels” , a mobile snack shop, to decide the sales spot/location within the  neighborhood of city of Toronto based on following criteria.  

  
    a.Top 10 Most Trending Venue ( Expect restaurants )  
    b.Nearest ( min distance)  from current sales spots  
    c.Venue Rating  
    d.Number of tips /  tips details  
    e.Consulting/contacting user who gave most valuable tip ( if needed)  
    f.Other popular spots near the venue  

#####  Let us help them to find the best sales spot !!!


#### Step  1:  Access & scrape the Wikipedia page , wrangle the data, clean it, and then read it into a pandas dataframe 

In [10]:
#Install wikipedia
!pip install wikipedia


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [11]:
import pandas as pd
import wikipedia as wp
 
#Get the html source
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Added Column header ,  Removed data with borough =  "Not assigned ".

In [12]:
df.columns = df.iloc[0]
df = df[1:]
df=df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [13]:
df['Neighbourhood'][df.Neighbourhood == 'Not assigned'] = df['Borough']

If more than one neighborhood  exist in one postal code area , combine them with  comma separated

In [14]:
df1 = df.groupby([('Postcode') , ('Borough')])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
# Rename Column name Postcode to PostalCode for Consistency
df1.rename(columns={'Postcode':'PostalCode'}, inplace=True)
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Step  2:   Access Geospatial Data for latitude and longitude information

In [15]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Appending Geospatial Data with dataframe based on postal code  ( .....df1.postalcode=df_geo.postalcode )

In [16]:
neighborhoods= (pd.merge(df1, df_geo, on='PostalCode'))
neighborhoods.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Import libraries for processing Json ,foursquare, matplot and folium

In [17]:
import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests

import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.5.1
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda




Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.5.1
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda




Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.




==> WARNING: A newer version of conda exists. <==
  current version: 4.5.1
  latest version: 4.5.11

Please update conda by running

    $ conda update -n base conda




Find the geograpical coordinate of Toronto

In [18]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


Draw a map of Toronto using latitude and longitude values

In [19]:

map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

####  Step 4 : Limit the scope of this project to Borough contain "Toronto".

In [20]:
Toronto_data = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
Toronto_data.head()


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
Toronto_data.shape

(38, 5)

####  Step 5 : Random selection of current location for test.

In [33]:
cur_loc=Toronto_data ['PostalCode'].sample()
latitude=Toronto_data['Latitude'].loc[Toronto_data['PostalCode'].isin(cur_loc)] .values[0]
longitude=Toronto_data['Longitude'].loc[Toronto_data['PostalCode'].isin(cur_loc)].values[0]
cur_Borough=Toronto_data['Borough'].loc[Toronto_data['PostalCode'].isin(cur_loc)].values[0]
print(latitude, longitude)
print (cur_Borough)

#latitude=43.653963
#longitude=-79.387207

latitude=40.7149555
longitude=-74.0153365



43.7280205 -79.3887901
Central Toronto


####  Step 6: Defining Foursqure credentials 

In [34]:
CLIENT_ID = 'YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5' # your Foursquare ID
CLIENT_SECRET = 'XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5
CLIENT_SECRET:XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD


#### Step 7: Accessing  Foursqure API for Explore Trending Venues

In [35]:
# Seraching within the radius of 2000 meters
radius = 2000
# limit the number of records to 50
LIMIT=100

#### Step 8: Sending request for trending location details & format result

In [36]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,  radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/trending?client_id=YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5&client_secret=XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD&ll=40.7149555,-74.0153365&v=20180605&radius=2000&limit=100'

In [37]:
results = requests.get(url).json()
results
x=len(results['response']['venues'])
x
if len(results['response']['venues']) == 0:
    dataframe= 'No trending venues are available at the moment!'
  
else:
     # assign relevant part of JSON to venues
        venues = results['response']['venues']

        # tranform venues into a dataframe
        dataframe = json_normalize(venues)
        dataframe.head()

dataframe 

,categories,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name
0,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",590a00e74febd55af43a8267,159 Mott St,US,New York,United States,Grand Street and Broom Street,1684,"[159 Mott St (Grand Street and Broom Street), ...","[{'label': 'display', 'lat': 40.71950256062705...",40.719503,-73.996297,10013,NY,Milk & Cream Cereal Bar


In [32]:
if len(results['response']['venues']) == 0:
    dataframe_filtered= 'No trending venues are available at the moment!'
else:
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]
    
    # function that extracts the category of the venue
    def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
            
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
    
    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

'No trending venues are available at the moment!'

#### Step 8 : Remove the data if  Category of Trending location contain "Restaurant"

In [371]:
if len(results['response']['venues']) == 0:
    dataframe_filtered= 'No trending venues are available at the moment!'
    
else:
    dataframe_filtered=dataframe_filtered[dataframe_filtered.categories != 'Restaurant']
    dataframe_filtered.head()
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Dante,Italian Restaurant,79-81 Macdougal St,US,New York,United States,btw Bleecker and W. Houston,1931,"[79-81 Macdougal St (btw Bleecker and W. Houston), New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.728847021360245, 'lng': -74.00162166908986}]",40.728847,-74.001622,NaN,10012,NY,556f4ef2498ee60139b28c72
1,Amazon 4-star,Miscellaneous Shop,72 Spring St,US,New York,United States,NaN,1694,"[72 Spring St, New York, NY 10012, United States]","[{'label': 'display', 'lat': 40.722424, 'lng': -73.99783}]",40.722424,-73.997830,SoHo,10012,NY,5bac05134b78c5002c40ac63
2,World Trade Center Transportation Hub (The Oculus),Metro Station,Liberty St & Church Ave,US,New York,United States,Fulton St,496,"[Liberty St & Church Ave (Fulton St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71159948119108, 'lng': -74.01146471500397}]",40.711599,-74.011465,NaN,10007,NY,4eb90d85722e09311d356915


#### Step 9: Presenting the data analysis result to "Snacks on Wheel" Business Owner

In [372]:
if len(results['response']['venues']) == 0:
    Trending_loc= 'No trending venues are available at the moment!'
else:
    Trending_loc = dataframe_filtered[['name', 'categories', 'address','city','distance','lat','lng']].copy()

print ('*************************************TRENDING VENUS ************************************************************')    
Trending_loc




*************************************TRENDING VENUS ************************************************************


,name,categories,address,city,distance,lat,lng
0,Dante,Italian Restaurant,79-81 Macdougal St,New York,1931,40.728847,-74.001622
1,Amazon 4-star,Miscellaneous Shop,72 Spring St,New York,1694,40.722424,-73.997830
2,World Trade Center Transportation Hub (The Oculus),Metro Station,Liberty St & Church Ave,New York,496,40.711599,-74.011465


In [373]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'
    
else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the trending location

    # add a red circle marker to represent the current location
    folium.features.CircleMarker(
    [latitude, longitude],
    radius=20,
    color='red',
    popup='Current Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(venues_map)
    
    # add the trending location as blue circle markers
    for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
        folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

###  Nearest Trending Venue Selection Based on Stat. Min distance

In [374]:
ShortPath=dataframe_filtered['distance'].idxmin()

print ('$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Nearest Trending Venue $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')

dataframe_filtered['name'].iloc[ShortPath]

$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Nearest Trending Venue $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


'World Trade Center Transportation Hub (The Oculus)'

#### Step 10 : Analyze the Trending Venue Further by Explore near by area.

In [375]:
venue_id = dataframe_filtered['id'].iloc[ShortPath]

url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4eb90d85722e09311d356915?client_id=YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5&client_secret=XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD&v=20180605'

In [376]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'venueRatingBlacklisted', 'beenHere', 'specials', 'photos', 'reasons', 'description', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4eb90d85722e09311d356915',
 'name': 'World Trade Center Transportation Hub (The Oculus)',
 'contact': {'facebook': '109423129134934',
  'facebookUsername': 'wtcprogress',
  'facebookName': 'WTC Progress'},
 'location': {'address': 'Liberty St & Church Ave',
  'crossStreet': 'Fulton St',
  'lat': 40.71159948119108,
  'lng': -74.01146471500397,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71159948119108,
    'lng': -74.01146471500397}],
  'postalCode': '10007',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['Liberty St & Church Ave (Fulton St)',
   'New York, NY 10007',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/world-trade-center-transportation-hub-the-oculus/4eb90d85722e09311d356915',
 'categories': [{'id': '4bf58dd8d48988d1fd931735',
   'name': 'Metro Station',
   'pluralName': 'Metro Stations',
   'shortName': 'Metro',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/trave

### Venue Rating 

In [377]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


###  Number of tips

In [378]:
result['response']['venue']['tips']['count']

72

### Venue's tips in details

In [379]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb12b044c1f6750c6bc6504'},
 'response': {'tips': {'count': 72,
   'items': [{'id': '585d6c6a00841a6cd0abb1f0',
     'createdAt': 1482517610,
     'text': "Great architecture, people watching, and shopping. Occasionally there will be special events. If you're going to check it out avoid rush hour as you will get trampled.",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/585d6c6a00841a6cd0abb1f0',
     'photo': {'id': '585d6c6ca598b313199e9f76',
      'createdAt': 1482517612,
      'source': {'name': 'Foursquare for iOS',
       'url': 'https://foursquare.com/download/#/iphone'},
      'prefix': 'https://igx.4sqi.net/img/general/',
      'suffix': '/13242960_Hkk6W4_voyN8WXZ2P6N7IZ7HwBhZ_ExIAPjsreF0y2A.jpg',
      'width': 1440,
      'height': 1920,
      'visibility': 'public'},
     'photourl': 'https://igx.4sqi.net/img/general/original/13242960_Hkk6W4_voyN8WXZ2P6N7IZ7HwBhZ_ExIAPjsreF0y2A.jpg',
     'lang': 'en',
     'likes

In [380]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'lastVoteText', 'lastUpvoteTimestamp', 'todo', 'user', 'authorInteractionType'])

In [381]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]


# display tips
print("############################ Venue's Tips###########################")

tips_filtered

############################ Venue's Tips###########################


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Great architecture, people watching, and shopping. Occasionally there will be special events. If you're going to check it out avoid rush hour as you will get trampled.",12,0,585d6c6a00841a6cd0abb1f0,Taiowa,Waner,male,13242960


### Analyze about the user who gave the tip 

In [382]:
user_id = tips_filtered['user.id'].iloc[0] # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()


dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

### User Name & Contact details for further details if needed

In [383]:
print('First Name: ' + user_data['firstName'])
print('Home City: ' + user_data['homeCity'])
print('contact: ' + str(user_data['contact']))

First Name: Taiowa
Home City: NYC
contact: {}


#### Step 11 :  Explore the location Further before final decision



In [384]:
latitude = dataframe_filtered['lat'].iloc[ShortPath]
longitude = dataframe_filtered['lng'].iloc[ShortPath]
LIMIT=20
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?client_id=YM4DEIRN1QRSQKLOKOMVW5LOFGR32DJPPSP2ASJOSPG5ISV5&client_secret=XWG2KZR0RV0PNUSC3JUWLQA3RFWPJBLRGTKC2V1BHNKDYSYD&ll=40.71159948119108,-74.01146471500397&v=20180605&radius=2000&limit=20'

In [385]:
import requests

In [386]:
results = requests.get(url).json()
'There are {} landmarks around this venue.'.format(len(results['response']['groups'][0]['items']))

'There are 20 landmarks around this venue.'

In [387]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b6b5abff964a520fb022ce3',
  'name': 'National September 11 Memorial & Museum',
  'location': {'address': '180 Greenwich St',
   'crossStreet': 'btwn Liberty & Fulton St',
   'lat': 40.71145106322093,
   'lng': -74.0134334564209,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71145106322093,
     'lng': -74.0134334564209}],
   'distance': 166,
   'postalCode': '10006',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['180 Greenwich St (btwn Liberty & Fulton St)',
    'New York, NY 10006',
    'United States']},
  'categories': [{'id': '5642206c498e4bfca532186c',
    'name': 'Memorial Site',
    'pluralName': 'Memorial Sites',
    'shortName': 'Memorial Site',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/historicsite_',

###   Popular Spots  Near BY Trending Venue

In [388]:
dataframe_E = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe_E.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_E = dataframe_E.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered_E['venue.categories'] = dataframe_filtered_E.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered_E.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

print ('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Popular Spots  Near BY Trending Venue Area @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')

dataframe_filtered_E.head(10)

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Popular Spots  Near BY Trending Venue Area @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,National September 11 Memorial & Museum,Memorial Site,180 Greenwich St,US,New York,United States,btwn Liberty & Fulton St,166,"[180 Greenwich St (btwn Liberty & Fulton St), New York, NY 10006, United States]","[{'label': 'display', 'lat': 40.71145106322093, 'lng': -74.0134334564209}]",40.711451,-74.013433,NaN,10006,NY,4b6b5abff964a520fb022ce3
1,One World Trade Center,Building,1 World Trade Ctr,US,New York,United States,btwn Fulton & West St,215,"[1 World Trade Ctr (btwn Fulton & West St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71306941116871, 'lng': -74.01313304901123}]",40.713069,-74.013133,NaN,10007,NY,4abe7f9ef964a520068e20e3
2,One World Observatory,Scenic Lookout,1 World Trade Ctr,US,New York,United States,Floor 100,204,"[1 World Trade Ctr (Floor 100), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.712819, 'lng': -74.013274}]",40.712819,-74.013274,NaN,10007,NY,515abd65e4b08eda86b30e6e
3,Four Seasons Hotel New York Downtown,Hotel,27 Barclay St,US,New York,United States,Broadway,208,"[27 Barclay St (Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71261246501983, 'lng': -74.00938032087628}]",40.712612,-74.009380,NaN,10007,NY,57c640ad498e74977f98372f
4,Nobu Downtown,Japanese Restaurant,195 Broadway,US,New York,United States,at Dey St.,197,"[195 Broadway (at Dey St.), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71053208431402, 'lng': -74.00959253311157}]",40.710532,-74.009593,NaN,10007,NY,4b60c708f964a520d0f829e3
5,Eataly Downtown,Market,101 Liberty St,US,New York,United States,btwn Church & Greenwich St,175,"[101 Liberty St (btwn Church & Greenwich St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.710075, 'lng': -74.011976}]",40.710075,-74.011976,NaN,10007,NY,579c12cd498e6e815b3acf50
6,9/11 Memorial North Pool,Fountain,West St,US,New York,United States,at Fulton St,154,"[West St (at Fulton St), New York, NY, United States]","[{'label': 'display', 'lat': 40.71207726282092, 'lng': -74.01318669319153}]",40.712077,-74.013187,NaN,NaN,NY,58801864cc5b6a14dedce689
7,9/11 Museum Auditorium,Auditorium,180 Greenwich St,US,New York,United States,NaN,117,"[180 Greenwich St, New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.711859516489525, 'lng': -74.01281437966045}]",40.711860,-74.012814,NaN,10007,NY,5941c0abf193c012d47fc4e8
8,Liberty Park,Park,Liberty St,US,New York,United States,at West St,243,"[Liberty St (at West St), New York, NY 10006, United States]","[{'label': 'display', 'lat': 40.71038391621699, 'lng': -74.01386844682942}]",40.710384,-74.013868,NaN,10006,NY,576c2212498e72c8c38f1ddd
9,The Wooly Daily,Coffee Shop,11 Barclay St,US,New York,United States,Broadway,265,"[11 Barclay St (Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71213724343698, 'lng': -74.00839457685005}]",40.712137,-74.008395,NaN,10007,NY,56093809498e5344ab8835a6


###  Map : Popular Spots  Near BY Trending Venue

In [392]:
 venues_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the trending location

# add a red circle marker to represent the current location
folium.features.CircleMarker(
    [latitude, longitude],
    radius=20,
    color='red',
    popup='Trending Venue-shortest distance',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(venues_map)
    
# add the trending location as blue circle markers
for lat, lng, label in zip(dataframe_filtered_E.lat, dataframe_filtered_E.lng, dataframe_filtered_E.categories):
        folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)


# display map
venues_map

## Now choose wisely, do you want to move to nearest trending location or continue here? We advise a move based on this analysis

### Thanks You - End of this Project
